<a href="https://colab.research.google.com/github/wolfinwallst/Financial_Engineering_and_Risk_Management/blob/main/Black_Scholes_binary_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[옵션 #7. 옵션 프리미엄 구하기 실습: Binomial Tree](https://sine-qua-none.tistory.com/113) 참조

1. 이항 트리를 사용해서 점화식을 유도하고, 이걸 이용해서 만기시 콜옵션 가격을 계산하고,
2. exact solution과 비교한다.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import time	# 계산에 소요된 시간을 분석하기 위해 필요한 time 모듈

N = norm.cdf

def CallOptionBS(S, K, T, r, q, sigma):
    if T == 0:
        return np.max(S - K, 0)
    else:
        d1 = (np.log(S / K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return S * np.exp(-q * T) * N(d1) - K * np.exp(-r * T) * N(d2)


def call_option_by_BinomialTree():
    s0 = 100
    vol = 0.3
    r = 0.02
    q = 0.01
    maturity = 1
    strike = 100
    nNode = 20
    dt = maturity / nNode

    u = np.exp(vol * np.sqrt(dt))	# 이항모델 세팅: 주가의 상승률
    d = 1 / u			# 이항모델 세팅: 주가의 하락률
    a = np.exp((r - q) * dt)
    p = (a - d) / (u - d)	# 주가의 상승확률

    def treeNode(n, i):	# binomial tree의 노드의 값들을 재귀적으로 계산하는 함수
        if n == nNode:	# 만기시에는,
            return np.max([s0 * u ** i * d ** (n - i) - strike, 0])	# 만기 payoff을 대입
            # return np.max([s0 * u ** (2 * i - n) - strike, 0])	# 만기 payoff을 u에 관해 정리한 식
        elif n < nNode:
            up_val = treeNode(n + 1, i + 1)
            down_val = treeNode(n + 1, i)
            df = np.exp(-r * dt)
            return (p * up_val + (1 - p) * down_val) * df	# 점화식을 사용하여 전단계 timestep의 값 산출

    start_time = time.time()	# 계산 시작 시점의 시각을 읽는다
    res = treeNode(0, 0)
    elapsed_time = time.time()-start_time	# 계산 끝난 시각(time.time())을 읽어 계산시작 시점을 뺀 값

    print('Binomial Tree price : {:.3f}'.format(res))
    print('exact         price : {:.3f}'.format(CallOptionBS(s0, strike, maturity, r, q, vol)))
    print('calculation time    : {:.2f}'.format(elapsed_time))

if __name__ == '__main__':
    call_option_by_BinomialTree()

Binomial Tree price : 12.099
exact         price : 12.245
calculation time    : 9.89
